<a href="https://colab.research.google.com/github/aruaru0/colab_notebook/blob/main/CV%E3%82%92%E8%AA%BF%E3%81%B9%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for coalab

In [1]:
!nvidia-smi

Sat Oct 23 22:15:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
check_only = True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install --upgrade --force-reinstall --no-deps  kaggle > /dev/null
! mkdir /root/.kaggle
! cp "/content/drive/My Drive/Kaggle/kaggle.json" /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

In [1]:
!kaggle competitions download -c chaii-hindi-and-tamil-question-answering
!kaggle datasets download -d nguyenduongthanh/xlm-roberta-large-squad-v2
!kaggle datasets download -d nbroad/xlm-roberta-squad2


chaii-hindi-and-tamil-question-answering.zip: Skipping, found more recently modified local copy (use --force to force download)
xlm-roberta-large-squad-v2.zip: Skipping, found more recently modified local copy (use --force to force download)
100% 2.53G/2.53G [01:13<00:00, 61.1MB/s]
100% 2.53G/2.53G [01:13<00:00, 36.8MB/s]


In [4]:
if not check_only : 
  !kaggle datasets download -d nguyenduongthanh/chaii-xlmr-5-fold
  !kaggle datasets download -d aruaru0/chaii3-1002
  !kaggle datasets download -d aruaru0/chaii-no006
  !kaggle datasets download -d aruaru0/chaii-no010

In [5]:
!apt install unzip
!mkdir -p input output

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [6]:
!mkdir input/chaii-hindi-and-tamil-question-answering
!unzip /content/chaii-hindi-and-tamil-question-answering.zip -d input/chaii-hindi-and-tamil-question-answering
!mkdir input/xlm-roberta-large-squad-v2
!unzip /content/xlm-roberta-large-squad-v2.zip -d input/xlm-roberta-large-squad-v2
!mkdir input/xlm-roberta-squad2
!unzip /content/xlm-roberta-squad2.zip -d input/xlm-roberta-squad2

Archive:  /content/chaii-hindi-and-tamil-question-answering.zip
  inflating: input/chaii-hindi-and-tamil-question-answering/sample_submission.csv  
  inflating: input/chaii-hindi-and-tamil-question-answering/test.csv  
  inflating: input/chaii-hindi-and-tamil-question-answering/train.csv  
Archive:  /content/xlm-roberta-large-squad-v2.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/xlm-roberta-large-squad-v2.zip or
        /content/xlm-roberta-large-squad-v2.zip.zip, and cannot find /content/xlm-roberta-large-squad-v2.zip.ZIP, period.
Archive:  /content/xlm-roberta-squad2.zip
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2/config.json  
  inflating: input/xlm-roberta-squad2/deepset/xlm-roberta-base-sq

In [7]:
if not check_only : 
  !mkdir input/chaii-xlmr-5-fold
  !unzip /content/chaii-xlmr-5-fold.zip -d input/chaii-xlmr-5-fold
  !rm /content/chaii-xlmr-5-fold.zip

  !mkdir input/chaii3-1002
  !unzip /content/chaii3-1002.zip -d input/chaii3-1002
  !rm /content/chaii3-1002.zip

  !mkdir input/chaii-no006
  !unzip /content/chaii-no006.zip -d input/chaii-no006
  !rm /content/chaii-no006.zip

  !mkdir input/chaii-no010
  !unzip /content/chaii-no010.zip -d input/chaii-no010
  !rm /content/chaii-no010.zip

In [8]:
!rm *.zip

# restart here...

In [9]:
%cd output

/content/output


In [30]:
!pip install transformers
!pip install optuna

     |████████████████████████████████| 308 kB 4.1 MB/s 
     |████████████████████████████████| 209 kB 60.2 MB/s 
     |████████████████████████████████| 80 kB 10.1 MB/s 
     |████████████████████████████████| 75 kB 5.0 MB/s 
     |████████████████████████████████| 111 kB 55.9 MB/s 
     |████████████████████████████████| 144 kB 49.3 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=a324429cbd1d7985fe9402d41974d0deeefbf7c8e9f76aadb6f122d48689abd2
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


# infer

In [11]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import gc
gc.enable()
import math
import json
import time
import random
import multiprocessing
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn import model_selection
from string import punctuation

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import torch.optim as optim
from torch.utils.data import (
    Dataset, DataLoader,
    SequentialSampler, RandomSampler
)
from torch.utils.data.distributed import DistributedSampler

try:
    from apex import amp
    APEX_INSTALLED = True
except ImportError:
    APEX_INSTALLED = False

import transformers
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    logging,
    MODEL_FOR_QUESTION_ANSWERING_MAPPING,
)
logging.set_verbosity_warning()
logging.set_verbosity_error()

def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def optimal_num_of_loader_workers():
    num_cpus = multiprocessing.cpu_count()
    num_gpus = torch.cuda.device_count()
    optimal_value = min(num_cpus, num_gpus*4) if num_gpus else num_cpus - 1
    return optimal_value

print(f"Apex AMP Installed :: {APEX_INSTALLED}")
MODEL_CONFIG_CLASSES = list(MODEL_FOR_QUESTION_ANSWERING_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

Apex AMP Installed :: False


In [12]:
class Config:
    # model
    model_type = 'xlm_roberta'
#     model_name_or_path = "../input/xlm-roberta-large-squad-v2"
#     config_name = "../input/xlm-roberta-large-squad-v2"
    model_name_or_path = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"
    config_name = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"
    fp16 = True if APEX_INSTALLED else False
    fp16_opt_level = "O1"
    gradient_accumulation_steps = 2

    # tokenizer
    tokenizer_name = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"
#     tokenizer_name = "../input/xlm-roberta-large-squad-v2"
    max_seq_length = 400
    doc_stride = 135

    # train
    epochs = 1
    train_batch_size = 4
    eval_batch_size = 135

    # optimzer
    optimizer_type = 'AdamW'
    learning_rate = 1e-9
    weight_decay = 1e-3
    epsilon = 1e-8
    max_grad_norm = 1.0

    # scheduler
    decay_name = 'linear-warmup'
    warmup_ratio = 0.1

    # logging
    logging_steps = 10

    # evaluate
    output_dir = 'output'
    seed = 2021

In [13]:
class DatasetRetriever(Dataset):
    def __init__(self, features, mode='train'):
        super(DatasetRetriever, self).__init__()
        self.features = features
        self.mode = mode
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, item):   
        feature = self.features[item]
        if self.mode == 'train':
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

In [14]:
class Model(nn.Module):
    def __init__(self, modelname_or_path, config):
        super(Model, self).__init__()
        self.config = config
        self.xlm_roberta = AutoModel.from_pretrained(modelname_or_path, config=config)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self._init_weights(self.qa_outputs)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
    ):
        outputs = self.xlm_roberta(
            input_ids,
            attention_mask=attention_mask,
        )

        sequence_output = outputs[0]
        pooled_output = outputs[1]
        
        # sequence_output = self.dropout(sequence_output)
        qa_logits = self.qa_outputs(sequence_output)
        
        start_logits, end_logits = qa_logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
        return start_logits, end_logits

In [15]:
def make_model(args):
    config = AutoConfig.from_pretrained(args.config_name)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
    model = Model(args.model_name_or_path, config=config)
    return config, tokenizer, model

In [16]:
def prepare_test_features(args, example, tokenizer):
    example["question"] = example["question"].lstrip()
    
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=args.max_seq_length,
        stride=args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    features = []
    for i in range(len(tokenized_example["input_ids"])):
        feature = {}
        feature["example_id"] = example['id']
        feature['context'] = example['context']
        feature['question'] = example['question']
        feature['input_ids'] = tokenized_example['input_ids'][i]
        feature['attention_mask'] = tokenized_example['attention_mask'][i]
        feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
        feature['sequence_ids'] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
        features.append(feature)
    return features

In [17]:
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]
        
        
    return predictions

In [18]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')

test['context'] = test['context'].apply(lambda x: ' '.join(x.split()))
test['question'] = test['question'].apply(lambda x: ' '.join(x.split()))

#base_model_path = '../input/chaii-qa-5-fold-xlmroberta-torch-fit'
# test=test[:10]
#test=test[:10]
tokenizer = AutoTokenizer.from_pretrained(Config().tokenizer_name)

test_features = []
for i, row in test.iterrows():
    test_features += prepare_test_features(Config(), row, tokenizer)

args = Config()
test_dataset = DatasetRetriever(test_features, mode='test')
test_dataloader = DataLoader(
    test_dataset,
    batch_size=args.eval_batch_size, 
    sampler=SequentialSampler(test_dataset),
    num_workers=optimal_num_of_loader_workers(),
    pin_memory=True, 
    drop_last=False
)

In [19]:
def get_predictions(checkpoint_path):
    config, tokenizer, model = make_model(Config())
    model.cuda();
    model.load_state_dict(
        torch.load(base_model + checkpoint_path)
    );
    
    start_logits = []
    end_logits = []
    for batch in test_dataloader:
        with torch.no_grad():
            outputs_start, outputs_end = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(outputs_start.cpu().numpy().tolist())
            end_logits.append(outputs_end.cpu().numpy().tolist())
            del outputs_start, outputs_end
    del model, tokenizer, config
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [20]:
if not check_only : 
  import pickle

  x = ["chaii-xlmr-5-fold","chaii3-1002","chaii-no006","chaii-no010"]
  for e in x:
    base_model = "../input/{}/output/".format(e)
    print(base_model)
    start_logits1, end_logits1 = get_predictions('checkpoint-fold-0/pytorch_model.bin')
    start_logits2, end_logits2 = get_predictions('checkpoint-fold-1/pytorch_model.bin')
    start_logits3, end_logits3 = get_predictions('checkpoint-fold-2/pytorch_model.bin')
    start_logits4, end_logits4 = get_predictions('checkpoint-fold-3/pytorch_model.bin')
    start_logits5, end_logits5 = get_predictions('checkpoint-fold-4/pytorch_model.bin')
    file = "{}.pkl".format(e)
    with open(file, 'wb') as f :
      pickle.dump(start_logits1, f)
      pickle.dump(end_logits1, f)
      pickle.dump(start_logits2, f)
      pickle.dump(end_logits2, f)
      pickle.dump(start_logits3, f)
      pickle.dump(end_logits3, f)
      pickle.dump(start_logits4, f)
      pickle.dump(end_logits4, f)
      pickle.dump(start_logits5, f)
      pickle.dump(end_logits5, f)


In [21]:
if not check_only : 
  !cp * /content/drive/MyDrive/datas/pkl

# read 

In [22]:
!cp /content/drive/MyDrive/datas/pkl/* .

In [23]:
import pickle

x = ["chaii-xlmr-5-fold","chaii3-1002","chaii-no006","chaii-no010"]

start = []
end = []
for e in x:
  file = "{}.pkl".format(e)
  with open(file, 'rb') as f :
    for _ in range(5) :
      start.append(pickle.load(f))
      end.append(pickle.load(f))

In [24]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')

def get_predictions(checkpoint_path):
    config, tokenizer, model = make_model(Config())
    model.cuda();
    model.load_state_dict(
        torch.load( checkpoint_path)
    );
    
    start_logits = []
    end_logits = []
    for batch in test_dataloader:
        with torch.no_grad():
            outputs_start, outputs_end = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(outputs_start.cpu().numpy().tolist())
            end_logits.append(outputs_end.cpu().numpy().tolist())
            del outputs_start, outputs_end
    del model, tokenizer, config
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [39]:
def objective(trial):
    n = len(start)
    samples = [0.2]*n

    for i in range(n):
      samples[i] = trial.suggest_int("w{}".format(i), 0, 1)

    print(samples)
    w = [samples[i]/sum(samples) for i in range(len(samples))]

    start_logits = np.zeros(start[0].shape)
    end_logits = np.zeros(start[0].shape)
    for i in range(n) :
      start_logits += w[i] * start[i]
      end_logits += w[i] * end[i]

    fin_preds = postprocess_qa_predictions(test, test_features, (start_logits, end_logits))

    submission = []
    for p1, p2 in fin_preds.items():
        p2 = " ".join(p2.split())
        p2 = p2.strip(punctuation)
        submission.append((p1, p2))

    sample = pd.DataFrame(submission, columns=["id", "PredictionString"])

    test_data =pd.merge(left=test,right=sample,on='id')


    bad_starts = [".", ",", "(", ")", "-", "–",  ",", ";"]
    bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

    tamil_ad = "கி.பி"
    tamil_bc = "கி.மு"
    tamil_km = "கி.மீ"
    hindi_ad = "ई"
    hindi_bc = "ई.पू"


    cleaned_preds = []
    for pred, context in test_data[["PredictionString", "context"]].to_numpy():
        if pred == "":
            cleaned_preds.append(pred)
            continue
        while any([pred.startswith(y) for y in bad_starts]):
            pred = pred[1:]
        while any([pred.endswith(y) for y in bad_endings]):
            if pred.endswith("..."):
                pred = pred[:-3]
            else:
                pred = pred[:-1]
        if pred.endswith("..."):
                pred = pred[:-3]

        if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
            pred = pred+"."

        cleaned_preds.append(pred)

    jac=[]    
    def jaccard(str1, str2): 
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))


    for i in range(len(test)):
        jac.append(jaccard(cleaned_preds[i], test.answer_text.values[i]))


    return np.mean(jac)

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-10-23 22:33:32,408] A new study created in memory with name: no-name-20c24737-3cf6-48ac-92ca-6ff84786f291


[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
Post-processing 1114 example predictions split into 14188 features.


[I 2021-10-23 22:33:38,284] Trial 0 finished with value: 0.5810711468364071 and parameters: {'w0': 1, 'w1': 1, 'w2': 1, 'w3': 1, 'w4': 1, 'w5': 0, 'w6': 1, 'w7': 0, 'w8': 1, 'w9': 0, 'w10': 0, 'w11': 0, 'w12': 0, 'w13': 1, 'w14': 1, 'w15': 1, 'w16': 0, 'w17': 0, 'w18': 0, 'w19': 0}. Best is trial 0 with value: 0.5810711468364071.


[1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]
Post-processing 1114 example predictions split into 14188 features.


[I 2021-10-23 22:33:44,074] Trial 1 finished with value: 0.5870391365566411 and parameters: {'w0': 1, 'w1': 1, 'w2': 0, 'w3': 1, 'w4': 1, 'w5': 0, 'w6': 1, 'w7': 1, 'w8': 1, 'w9': 0, 'w10': 1, 'w11': 1, 'w12': 1, 'w13': 0, 'w14': 1, 'w15': 1, 'w16': 1, 'w17': 0, 'w18': 1, 'w19': 1}. Best is trial 1 with value: 0.5870391365566411.


...

[1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0]
Post-processing 1114 example predictions split into 14188 features.


[I 2021-10-23 22:38:54,505] Trial 54 finished with value: 0.5919567080137098 and parameters: {'w0': 1, 'w1': 0, 'w2': 0, 'w3': 0, 'w4': 1, 'w5': 1, 'w6': 0, 'w7': 1, 'w8': 0, 'w9': 1, 'w10': 1, 'w11': 1, 'w12': 1, 'w13': 1, 'w14': 0, 'w15': 1, 'w16': 1, 'w17': 0, 'w18': 1, 'w19': 0}. Best is trial 49 with value: 0.5919567080137098.


[1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0]
Post-processing 1114 example predictions split into 14188 features.


[I 2021-10-23 22:39:00,723] Trial 55 finished with value: 0.5919567080137098 and parameters: {'w0': 1, 'w1': 0, 'w2': 0, 'w3': 0, 'w4': 1, 'w5': 1, 'w6': 0, 'w7': 1, 'w8': 0, 'w9': 1, 'w10': 1, 'w11': 1, 'w12': 1, 'w13': 1, 'w14': 0, 'w15': 1, 'w16': 1, 'w17': 0, 'w18': 1, 'w19': 0}. Best is trial 49 with value: 0.5919567080137098.


[1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0]
